# Inference 
Inference in deep learning is the process of using a trained model to make predictions on new, unseen data. It involves applying the learned parameters without further updates to generate outputs such as classifications or predictions.

## Setup and Imports

In [1]:
# import dependencies
from IPython.display import display

from deepEM.Utils import create_text_widget

from src.Inferencer import Inference



## 3.1. Define Data

In difference to other deep learning methods, we will need to train a model also during inference. This is the case due to the special nature of the approach: It is "overfitting" the model to the reconstructed sample itself. 

Hence, for inference please provide a tilt series with following data structure: 

The data needs to be organized in a single folder containing a series of `.tif` files, which are sorted by their names. 

The folder further needs to contain a `.rawtlt` file with the tilt angles of the sinlge micrographs. The tilt angles are sorted based on the name sorting of the `.tif` files. When the data is currently in `.mrc` file format, you can generate a folder containing `.tif` images by using the `mrc2tif` command of the [iMOD](https://bio3d.colorado.edu/imod/) software or by using the [ImageJ](https://imagej.net/ij/) software. 

Lastly, we require a `metadata.json` with following content:

```json 
{
    "slice_thickness_nm": 550,
    "pixelsize_nmperpixel": 1.0,
    "original_px_resolution": 1000
}
```

 - `slice_theickness_nm` is the approximated slice thickness of your sample in [nm]. If you are uncertain, you should define an upper bound. 
 - `pixelsize_nmperpixel` is the pixelsize of your dataset in [nm/px]. 
 - `original_px_resolution` is the image resolution of a single `.tif` in your tilt series in [px]. 


You can generate such a `metadata.json` file, within any text editor of your choice. Add the above lines as content and adapt the parameters based on your data. Save the file as `metadata.json`. 

An example with a tilt series of five EM images and the corresponding `.rawtlt` and `metadata.json` is shown below: 

```
data/
├── image_001.tif
├── image_002.tif
├── image_003.tif
├── image_004.tif
├── image_005.tif
├── metadata.json
└── tilts.rawtlt

```


In [ ]:
data_widget = create_text_widget("Data Path:","data/real/noisy-projections", "Enter the path to a directory which contains .tif files you wish to do inference on.")
batch_widget = create_text_widget("Batch Size:", 16, "Please set the batch size for inference. Larger batch size can lead to faster computation but may lead to OOM (out of memory) errors.")
resize_widget = create_text_widget("Resize:", 100, "Downsizing the tilt series for faster model training. However, note that stonger downscaling leads to stronger loss of information..")


display(*data_widget)
display(*batch_widget)
display(*resize_widget)




In [3]:
data_path = data_widget[0].value
batch_size = int(batch_widget[0].value)
resize = int(resize_widget[0].value)

## 3.2. Choose Model

As mentioned above, due to the special nature of this deep learning approach, we require to retrain the model during inference. 
Still, we need to retrieve the best set of hyperparameters from our previous model developlent. 
These parameters were saved together with the model weights during the training.
Hence, we are loading the model checkpoint.

In [ ]:
model_widget = create_text_widget("Model Path:","logs/synthetic_2025-03-25_17-49-28", "Enter the path to a pretrained model which you'd like to use for inference.")
display(*model_widget)

## 3.3. Make Prediction
By executing the cell below, your trained model will be used to make predictions on the defined data. The predictions will be saved inside the folder of your data at `<currentdatetime>`.  

In [ ]:
model_path = model_widget[0].value
inferencer = Inference(model_path, data_path, batch_size, resize)
inferencer.inference()